In [39]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [40]:
import requests
import csv
from bs4 import BeautifulSoup
import pandas as pd
import re
import os

In [41]:
page = requests.get('https://www.royal.uk/speeches')

In [42]:
# Create a BeautifulSoup object
soup_main = BeautifulSoup(page.text, 'html.parser')

In [43]:
#get the list of members of the royal house
list_of_members=[]
list_complete=soup_main.find(class_="form-item form-type-select form-item-mrf").find_all('option')
for i in list_complete:
    list_of_members.append((i.text, i['value']))
list_of_members.pop(0)

('All Members of the Royal Family', '')

In [44]:
#I create a list with all the links to the speeches appearing when searching for each family member on the official Royal Family Website
for i in list_of_members:
    name=re.sub(' ', '', i[0])
    if not os.path.exists('speeches/'+name):
        os.makedirs('speeches/'+name)
    page=requests.get('https://www.royal.uk/speeches?text=&mrf='+i[1]+'&date[min][date]=1%2F01%2F1970&date[max][date]=13%2F06%2F2022&id=')
    soup=BeautifulSoup(page.text, 'html.parser')
    if soup.find(class_="pager-last last") is not None:
        nb_last_page=soup.find(class_="pager-last last").find('a').get('href').split('page=')[1]
    else:
        nb_last_page=0
    #working with the first page
    list_of_links_with_date=[]
    link_infos= soup.find_all(class_='tbm-fields-container views-fieldset')
    for item in link_infos:
        link_ref=item.find('a')
        link = 'https://www.royal.uk' + link_ref.get('href')
        date=item.find(class_="date-display-single")
        year=date.text.split()[-1]
        list_of_links_with_date.append((link, year))
    #if there is more than one page, I work with the other pages
    if nb_last_page!=0:
        list_of_pages=[]
        for j in range(1, int(nb_last_page)+1):
            list_of_pages.append('https://www.royal.uk/speeches?text=&mrf='+i[1]+'&date[min][date]=01/01/1970&date[max][date]=13/06/2022&id=&page='+str(j)) 
        for m in list_of_pages:
            page = requests.get(m)
            soup = BeautifulSoup(page.text, 'html.parser')
            link_infos= soup.find_all(class_='tbm-fields-container views-fieldset')
            for item in link_infos:
                link_ref=item.find('a')
                link = 'https://www.royal.uk' + link_ref.get('href')
                date=item.find(class_="date-display-single")
                year=date.text.split()[-1]
                list_of_links_with_date.append((link, year))
    #I print the list of links of the articles+year for every member of the Royal family
    file = open('speeches/'+name+'/'+name+'links_and_dates.csv', 'w')
    f=csv.writer(file)
    for l in range(len(list_of_links_with_date)):
        f.writerow([list_of_links_with_date[l][0], list_of_links_with_date[l][1]])
    file.close()          
        


In [45]:
#I retrieve all the speeches following the available links and print the links that appear to be broken
for i in list_of_members:
    name=re.sub(' ', '', i[0])
    file_links='speeches/'+name+'/'+name+'links_and_dates.csv'
    if os.stat(file_links).st_size != 0:
        df=pd.read_csv(file_links, names=["link", "year"])
        for index, row in df.iterrows():
            title='x'
            content='x'
            page = requests.get(row["link"], allow_redirects=False) #This is necessary because of the fact that some links are broken
            if page.status_code == 200:
                soup = BeautifulSoup(page.text, 'html.parser')
                last_links = soup.find(class_="tbm-share-links share")
                last_links.decompose()
                if soup.find(class_="container article").find('h1') is not None:
                    title=re.sub('[^0-9a-zA-Z]+', '', soup.find(class_="container article").find('h1').text)
                if soup.find(class_="block block-system").find(class_="content") is not None:
                    if soup.find(class_="block block-system").find(class_="content").find('blockquote') is not None:
                        content_block= soup.find(class_="block block-system").find(class_="content")
                        content_block.find('blockquote').decompose()
                        content=content_block.find_all('p')
                    else:
                        content= soup.find(class_="block block-system").find(class_="content").find_all('p')
                    content_text=''
                    for j in content[:-1]:
                        content_text+=j.text
                f=open('speeches/'+name+'/'+name+'_'+str(row["year"])+'_'+title+'.txt', 'w', encoding="utf-8")
                f.write(content_text)
                f.close()
            else:
                print(row["link"])


https://www.royal.uk/speech-following-death-diana-princess-wales
https://www.royal.uk/speech-following-death-diana-princess-wales
https://www.royal.uk/speech-following-death-diana-princess-wales
https://www.royal.uk/speech-following-death-diana-princess-wales


In [46]:
#I manually add the speech for the death of Diana (broken link)
page = requests.get("https://www.royal.uk/queens-message-following-death-diana-princess-wales")
soup = BeautifulSoup(page.text, 'html.parser')
last_links = soup.find(class_="tbm-share-links share")
last_links.decompose()
if soup.find(class_="container article").find('h1') is not None:
    title=re.sub('[^0-9a-zA-Z]+', '', soup.find(class_="container article").find('h1').text)
if soup.find(class_="block block-system").find(class_="content") is not None:
    if soup.find(class_="block block-system").find(class_="content").find('blockquote') is not None:
        content_block= soup.find(class_="block block-system").find(class_="content")
        content_block.find('blockquote').decompose()
        content=content_block.find_all('p')
    else:
        content= soup.find(class_="block block-system").find(class_="content").find_all('p')
    content_text=''
    for j in content[:-1]:
        content_text+=j.text
f=open('speeches/HerMajestyTheQueen/HerMajestyTheQueen_1997_'+title+'.txt', 'w', encoding="utf-8")
f.write(content_text)
f.close()

In [47]:
#I manually add the Christmas Broadcasts before 1970 (not appearing in the search)
page = requests.get("https://www.royal.uk/queens-first-christmas-broadcast-1952")
soup = BeautifulSoup(page.text, 'html.parser')
last_links = soup.find(class_="tbm-share-links share")
last_links.decompose()
if soup.find(class_="container article").find('h1') is not None:
    title=re.sub('[^0-9a-zA-Z]+', '', soup.find(class_="container article").find('h1').text)
if soup.find(class_="block block-system").find(class_="content") is not None:
    if soup.find(class_="block block-system").find(class_="content").find('blockquote') is not None:
        content_block= soup.find(class_="block block-system").find(class_="content")
        content_block.find('blockquote').decompose()
        content=content_block.find_all('p')
    else:
        content= soup.find(class_="block block-system").find(class_="content").find_all('p')
    content_text=''
    for j in content[:-1]:
        content_text+=j.text
f=open('speeches/HerMajestyTheQueen/HerMajestyTheQueen_1952_'+title+'.txt', 'w', encoding="utf-8")
f.write(content_text)
f.close()
for i in range (1953, 1970):
    page = requests.get("https://www.royal.uk/christmas-broadcast-"+str(i))
    soup = BeautifulSoup(page.text, 'html.parser')
    last_links = soup.find(class_="tbm-share-links share")
    last_links.decompose()
    if soup.find(class_="container article").find('h1') is not None:
        title=re.sub('[^0-9a-zA-Z]+', '', soup.find(class_="container article").find('h1').text)
    if soup.find(class_="block block-system").find(class_="content") is not None:
        content= soup.find(class_="block block-system").find(class_="content").find_all('p')
        content_text=''
        for j in content[:-1]:
            content_text+=j.text
    f=open('speeches/HerMajestyTheQueen/HerMajestyTheQueen_'+str(i)+'_'+title+'.txt', 'w', encoding="utf-8")
    f.write(content_text)
    f.close()

1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
